# Microsoft Autogen


In [ ]:
from autogen_core.models import UserMessage
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_core.models import ModelInfo

model_client = OpenAIChatCompletionClient(
    model="gemini-2.5-flash",
    model_info=ModelInfo(vision=True, function_calling=True, json_output=True, family="unknown", structured_output=True),
    api_key="APIKEY",
)

response = await model_client.create([UserMessage(content="What is the capital of France?", source="user")])
print(response)

finish_reason='stop' content='The capital of France is **Paris**.' usage=RequestUsage(prompt_tokens=8, completion_tokens=8) cached=False logprobs=None thought=None


In [39]:
from autogen_agentchat.agents import AssistantAgent
#Define a simple function tool that the agent can use.
# For this example, we use a fake weather tool for demonstration purposes.
async def get_weather(city: str) -> str:
    """Get the weather for a given city."""
    return f"The weather in {city} is 73 degrees and Sunny."


# Define an AssistantAgent with the model, tool, system message, and reflection enabled.
# The system message instructs the agent via natural language.
agent = AssistantAgent(
    name="weather_agent",
    model_client=model_client,
    tools=[get_weather],
    system_message="You are a helpful assistant.",
    reflect_on_tool_use=True,
    model_client_stream=True,  # Enable streaming tokens from the model client.
)


In [ ]:
from autogen_agentchat.ui import Console
# Run the agent and stream the messages to the console.
async def main() -> None:
    await Console(agent.run_stream(task="What is the weather in New York?"))

In [13]:
# NOTE: if running this inside a Python script you'll need to use asyncio.run(main()).
await main()

---------- TextMessage (user) ----------
What is the weather in New York?
---------- ToolCallRequestEvent (weather_agent) ----------
[FunctionCall(id='function-call-1220718225316406155', arguments='{"city":"New York"}', name='get_weather')]
---------- ToolCallExecutionEvent (weather_agent) ----------
[FunctionExecutionResult(content='The weather in New York is 73 degrees and Sunny.', name='get_weather', call_id='function-call-1220718225316406155', is_error=False)]
---------- ModelClientStreamingChunkEvent (weather_agent) ----------
The weather in New York is 73 degrees and Sunny.


In [15]:
import asyncio

from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.base import TaskResult
from autogen_agentchat.conditions import ExternalTermination, TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_core import CancellationToken
from autogen_ext.models.openai import OpenAIChatCompletionClient


In [40]:
# Create the primary agent.
primary_agent = AssistantAgent(
    "primary",
    model_client=model_client,
    system_message=(
        "You are the main AI assistant responsible for generating helpful, accurate, and well-structured responses "
        "to user queries."
    ),
)

# Create the critic agent.
critic_agent = AssistantAgent(
    "critic",
    model_client=model_client,
    system_message=(
        "You are a critical reviewer. Analyze the primary agent’s response and provide clear, constructive feedback "
        "to improve accuracy, clarity, and tone. "
        "Once the feedback has been fully addressed, respond with 'PASS'."
    ),
)

# Create the eye agent.
eye_agent = AssistantAgent(
    "eye",
    model_client=model_client,
    system_message=(
        "You are the overseer of the critic. Evaluate whether the critic's feedback is fair, relevant, and helpful. "
        "If the critic provides valid and balanced feedback, respond with 'APPROVE'. "
        "If not, suggest how the critic can improve their review."
    ),
)


In [ ]:
# Define a termination condition that stops the task if the critic approves.
text_termination = TextMentionTermination("APPROVE")

# Create a team with the primary and critic agents.
team = RoundRobinGroupChat([primary_agent, critic_agent,eye_agent], termination_condition=text_termination)

In [1]:
# Use `asyncio.run(...)` when running in a script.
result = await team.run(task="Write a short poem about the fall season.")
result


NameError: name 'team' is not defined

In [ ]:
# When running inside a script, use a async main function and call it from `asyncio.run(...)`.
await team.reset()  # Reset the team for a new task.
async for message in team.run_stream(task="Write a short poem about the fall season."):  # type: ignore
    if isinstance(message, TaskResult):
        print("Stop Reason:", message.stop_reason)
    else:
        print(message)
        

id='0da04372-95a3-445a-8586-f3b876eb14e7' source='user' models_usage=None metadata={} created_at=datetime.datetime(2025, 10, 8, 19, 19, 2, 557665, tzinfo=datetime.timezone.utc) content='Write a short poem about the fall season.' type='TextMessage'
id='fd79db96-eb94-4628-873a-772f9933ae2e' source='primary' models_usage=RequestUsage(prompt_tokens=33, completion_tokens=32) metadata={} created_at=datetime.datetime(2025, 10, 8, 19, 19, 6, 989396, tzinfo=datetime.timezone.utc) content="Golden leaves begin to fall,\nA crisp wind whispers through it all.\nPumpkins glow, a harvest moon,\nWinter's quiet coming soon." type='TextMessage'
id='a923b886-0632-42f9-8128-cade6477054e' source='critic' models_usage=RequestUsage(prompt_tokens=85, completion_tokens=371) metadata={} created_at=datetime.datetime(2025, 10, 8, 19, 19, 16, 750092, tzinfo=datetime.timezone.utc) content='This is a pleasant and clear short poem that effectively captures some classic elements of the fall season.\n\n**Strengths:**\n*